In [3]:
    #import Pkg
    #packages = [\"CSV\",\"DataFrames\",\"PlutoUI\",\"Plots\",\"Combinatorics\"]   \n",
    #Pkg.add(packages)\n",
   
    using CSV, DataFrames, PlutoUI, Plots, Combinatorics
    begin
        path = "/home/jonathan/Datasets/iris.data"
        csv_data = CSV.File(path, header=false)
        y  =  vcat(-(ones(50)),ones(50),-(ones(50)))
        iris_names = ["sepal_len", "sepal_wid", "petal_len", "petal_wid", "output","class"]
        df = DataFrame([csv_data.Column1,csv_data.Column2,csv_data.Column3,csv_data.Column4,y,csv_data.Column5], Symbol.(iris_names))
        dropmissing!(df)
    end
    
    
    using Flux
    using Flux: Data.DataLoader

    using Random
        using IterTools: ncycle

        Random.seed!(123);

    begin   
        # Convert df to array
        data = Matrix(df)

        # Shuffle
        data = data[shuffle(1:end), :]

        # train/test split
        train_test_ratio = .7
        idx = Int(floor(size(df, 1) * train_test_ratio))
        data_train = data[1:idx,:]
        data_test = data[idx+1:end, :]

        tuple_train = []
        tuple_test = []

        #tuple_convert(d::Array) = (d[:,1:4], d[:,5])
        for d = 1: size(data_train)[1]
            append!(tuple_train, [(data_train[d,1:4],data_train[d,5])])
        end
        for d = 1:size(data_test)[1]
           append!(tuple_test, [(data_test[d,1:4],data_test[d,5])])
        end

    end
    

In [4]:
#Function Like Objects
struct Perceptron{T}
    W::Vector{T}
    sigma
end
function (p::Perceptron)(x::Vector)
    return p.sigma(p.W'*x) 
end

mutable struct PerceptronOpt
    eta::Real
end

p = Perceptron{Real}(rand(4),x->sign(x))

opt = PerceptronOpt(1)

function update_w!(p::Perceptron,opt,data::Array)
    for d in data
        x = d[1]
        y = d[2]
        for i = 1:length(p.W)
            p.W[i]= p.W[i] + opt.eta*(y- p(x))*x[i]
        end
    end
end
@show p

p = Perceptron{Real}(Real[0.4448473974235778, 0.8708043872171827, 0.20752376940059802, 0.13752218436607366], var"#5#6"())


Perceptron{Real}(Real[0.4448473974235778, 0.8708043872171827, 0.20752376940059802, 0.13752218436607366], var"#5#6"())

In [5]:
tuple_train = []
tuple_test = []
#tuple_convert(d::Array) = (d[:,1:4], d[:,5])
for d = 1: size(data_train)[1]
    append!(tuple_train, [(data_train[d,1:4],data_train[d,5])])
end

for d = 1:size(data_test)[1]
    append!(tuple_test, [(data_test[d,1:4],data_test[d,5])])
end



In [6]:

function update_confusion_matrix!(cm, error, sum)
    #update false predictions
    if error > 0
        cm[2][1] += 1
    elseif error < 0 
        cm[1][2] += 1
    end
    if sum > 0
        cm[1][1] += 1
    elseif sum < 0 
        cm[2][2] += 1
    end
end
function get_classif_accuracy(cm)
    TP = cm[1][1]
    TN = cm[2][2]
    P = cm[1][1] + cm[1][2]
    N = cm[2][1] + cm[2][2]
    return TP + TN/(P+N)
end
    

function get_classif_accuracy(cm)
        TP = cm[1][1]
        TN = cm[2][2]
        P = cm[1][1] + cm[1][2]
        N = cm[2][1] + cm[2][2]
        return (TP + TN)/(P+N)
end



get_classif_accuracy (generic function with 1 method)

In [102]:
using BenchmarkTools
#train
data_size = 10
#   TP  FP  FN  TN
cm = [[0   0],[0   0]] 

@time update_w!(p,opt,tuple_train[1:data_size])


for n = 1:length(tuple_test)[1]
        error = tuple_test[n][2] - p(tuple_test[n][1])
        sum = tuple_test[n][2] + p(tuple_test[n][1])
        update_confusion_matrix!(cm, error, sum)
end

@show cm
get_classif_accuracy(cm)

  0.000050 seconds (572 allocations: 9.094 KiB)
cm = [[0 1], [14 30]]


0.6666666666666666

## Data Size  = 10
* time = 0.000054
* cm = [[1 2], [13 29]]
* accuracy = 0.6666666666666666
## Data Size  = 50
* time = 0.000151 
* cm = [[14 16], [0 15]]
* accuracy = 14.244444444444444
## Data Size  = 105
* time = 0.000274
* cm = [[4 12], [10 19]]
* accuracy = 


In [ ]:
p.W